In [1]:
import os
import gc
import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from scipy import sparse
from scipy.sparse.linalg import spsolve
from joblib import load, dump
from tqdm import tqdm
from scipy.signal import butter, lfilter, medfilt

# %matplotlib qt
sns.set(font_scale=1.5)
sns.set_style("white", {'font.family':'serif', 'font.serif':'Times New Roman'})

# Constants

In [2]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_2 = 'gait-dbase-2/'

N_SAMPLES = 1024
FORCE_THRESHOLD = 20
TOE_HEIGTH_THRESHOLD = 40

# Filter

In [3]:
class LowPassFilter(object): 
    @staticmethod
    def butter_lowpass(cutoff, fs, order):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def apply(data, cutoff=6, fs=2000, order=2, axis=-1):
        b, a = LowPassFilter.butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data, axis=axis)
        y = y - np.median(y)
        return y

# Function Defs

In [4]:
def standardize(x):
    # return (x - np.mean(x, axis=0)) / np.std(x, axis=0)
    return (x - np.min(x, axis=0)) / (np.max(x, axis=0) - np.min(x, axis=0))

def require_positive(x):
    if x < 0:
        return 0
    else:
        return x

def clip_overflow(x, max=N_SAMPLES):
    if x >= max:
        return N_SAMPLES - 1
    else:
        return x

def clip_noise(x, min=FORCE_THRESHOLD):
    mask = x > min
    new_x = np.zeros(x.shape)
    new_x[mask] = x[mask]
    return new_x

def get_first_grf(x, swing_period):
    half_way = x.shape[0] / 2
    temp = np.zeros(x.shape)
    temp[:half_way] = x[:half_way]
    stance_mask = temp < FORCE_THRESHOLD
    stance_start = np.min(stance_mask == True)
    stance_end = np.max(stance_mask == True)
    roi_start = stance_start
    roi_end = stance_end + swing_period
    return temp[roi_start : roi_end]

def get_processed_grf(grf, clip=False):
    processed_grf = LowPassFilter.apply(grf, fs=2000, cutoff=15)
    if clip:
        processed_grf[processed_grf < FORCE_THRESHOLD] = 0
    return resample(processed_grf, N_SAMPLES * 4, axis=0).T

def get_processed_moment(moment):
    processed_moment = LowPassFilter.apply(moment, fs=2000, cutoff=15)
    return resample(processed_moment, N_SAMPLES * 4, axis=0).T
    

In [5]:
# Use trajectories-z 6-9 for y-axis

def get_trajectories(content, foot='right'):
    trajectory_x, trajectory_y = (None, None)
    if foot == 'right':
        # trajectory_x = content['data']['points'][0, 18:23, :]
        # trajectory_y = content['data']['points'][1, 18:23, :]
        trajectory_z = content['data']['points'][2, 18:23, :]

    elif foot == 'left':
        # trajectory_x = content['data']['points'][0, 6:11, :]
        # trajectory_y = content['data']['points'][1, 6:11, :]
        trajectory_z = content['data']['points'][2, 6:11, :]

    else:
        raise ValueError('Third foot is not allowed')

    # trajectory_x = LowPassFilter.apply(trajectory_x, fs=200, cutoff=6, order=4)
    # trajectory_y = LowPassFilter.apply(trajectory_y, fs=200, cutoff=6, order=4)
    trajectory_z = LowPassFilter.apply(trajectory_z, fs=200, cutoff=6, order=4)

    # trajectory_x = resample(trajectory_x, N_SAMPLES * 4, axis=1).T
    # trajectory_y = resample(trajectory_y, N_SAMPLES * 4, axis=1).T
    trajectory_z = resample(trajectory_z, N_SAMPLES * 4, axis=1).T

    return trajectory_x, trajectory_y, trajectory_z

def get_force(content):
    force_x = []
    force_y = []
    force_z = []
    for i in range(1, 5):
        force_x.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][0, :]
            )
        )
        force_y.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][1, :]
            )
        )
        force_z.append(
            get_processed_grf(
                content['data']['platform'][i]['force'][2, :], True
            )
        )

    return force_x, force_y, force_z

def get_moment(content):
    moment_x = []
    moment_y = []
    moment_z = []
    for i in range(1, 5):
        moment_x.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][0, :]
            )
        )
        moment_y.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][1, :]
            )
        )
        moment_z.append(
            get_processed_moment(
                content['data']['platform'][i]['moment'][2, :]
            )
        )

    return moment_x, moment_y, moment_z

def get_swing_period(force_plate_data_z):
    grf_combined_z_r = force_plate_data_z[0] + force_plate_data_z[2]
    grf_combined_z_l = force_plate_data_z[1] + force_plate_data_z[3]

    grf_mask_r = (grf_combined_z_r >= FORCE_THRESHOLD)
    grf_mask_l = (grf_combined_z_l >= FORCE_THRESHOLD)

    grf_start_r = np.min(np.argwhere(grf_mask_r == True))
    grf_end_r = np.max(np.argwhere(grf_mask_r == True))
    grf_start_l = np.min(np.argwhere(grf_mask_l == True))
    grf_end_l = np.max(np.argwhere(grf_mask_l == True))

    swing_period_r = np.sum(grf_combined_z_r[grf_start_r:grf_end_r] <= FORCE_THRESHOLD)
    swing_period_l = np.sum(grf_combined_z_l[grf_start_l:grf_end_l] <= FORCE_THRESHOLD)

    return swing_period_r, swing_period_l
    

# Test Bench

In [ ]:
path = '/home/ai/Python/GRF-Synthesis-from-Motion-Trajectories/Dataset/gait-dbase-2/Participant3/Raw_Data/V35/T1.c3d'

In [ ]:
content = c3d(path, extract_forceplat_data=True)

# Extract Features

In [6]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_2)
subjects = os.listdir(path)

# subjects = subjects[:8]

features = pd.DataFrame()
target = pd.DataFrame()

records = []
n_samples = 0
n_bad_samples = 0

# ax1 = plt.axes(projection='3d')

for subject in tqdm(subjects):
    velocities_path = os.path.join(path, subject, "Raw_Data")
    velocities = os.listdir(velocities_path)

    for velocity in velocities:

        # if 'V1' in velocity or 'V15' in velocity or 'V2' in velocity or 'V25' in velocity:
        #     continue

        files_path = os.path.join(velocities_path, velocity)
        files = os.listdir(files_path)

        for filename in files:
            try:
                content = c3d(
                    os.path.join(files_path, filename), extract_forceplat_data=True
                )

                trajectory_x_r, trajectory_y_r, trajectory_z_r = get_trajectories(
                    content, "right"
                )
                trajectory_x_l, trajectory_y_l, trajectory_z_l = get_trajectories(
                    content, "left"
                )

                # ax1.plot3D(trajectory_y_r[:, 3], trajectory_x_r[:, 3], trajectory_z_r[:, 3])

                force_x, force_y, force_z = get_force(content)
                moment_x, moment_y, moment_z = get_moment(content)
                swing_period_r, swing_period_l = get_swing_period(force_z)

                del content
                gc.collect()

                mask_force_z_r_1 = force_z[0] > FORCE_THRESHOLD
                mask_force_z_r_2 = force_z[2] > FORCE_THRESHOLD
                mask_force_z_l_1 = force_z[1] > FORCE_THRESHOLD
                mask_force_z_l_2 = force_z[3] > FORCE_THRESHOLD

                # ----------------------- RIGHT ------------------------

                start = np.min(np.argwhere(mask_force_z_r_1 == True))
                end = np.max(np.argwhere(mask_force_z_r_1 == True))  # + swing_period_r

                force_x_r_1 = -force_x[0][start:end]
                force_y_r_1 = force_y[0][start:end]
                force_z_r_1 = force_z[0][start:end]

                moment_x_r_1 = moment_x[0][start:end]
                moment_y_r_1 = moment_y[0][start:end]
                moment_z_r_1 = moment_z[0][start:end]

                # trajectory_x_r_1 = trajectory_x_l[start:end]
                # trajectory_y_r_1 = trajectory_y_r[start:end]
                trajectory_z_r_1 = trajectory_z_r[start:end]

                start = np.min(np.argwhere(mask_force_z_r_2 == True))
                end = np.max(np.argwhere(mask_force_z_r_2 == True))  # + swing_period_r

                force_x_r_2 = -force_x[2][start:end]
                force_y_r_2 = force_y[2][start:end]
                force_z_r_2 = force_z[2][start:end]

                moment_x_r_2 = moment_x[2][start:end]
                moment_y_r_2 = -moment_y[2][start:end]
                moment_z_r_2 = -moment_z[2][start:end]

                # trajectory_x_r_2 = trajectory_x_l[start:end]
                # trajectory_y_r_2 = trajectory_y_r[start:end]
                trajectory_z_r_2 = trajectory_z_r[start:end]

                # ----------------------- LEFT ------------------------

                start = np.min(np.argwhere(mask_force_z_l_1 == True))
                end = np.max(np.argwhere(mask_force_z_l_1 == True))  # + swing_period_l

                force_x_l_1 = force_x[1][start:end]
                force_y_l_1 = force_y[1][start:end]
                force_z_l_1 = force_z[1][start:end]

                moment_x_l_1 = moment_x[1][start:end]
                moment_y_l_1 = moment_y[1][start:end]
                moment_z_l_1 = moment_z[1][start:end]

                # trajectory_x_l_1 = trajectory_x_r[start:end]
                # trajectory_y_l_1 = trajectory_y_l[start:end]
                trajectory_z_l_1 = trajectory_z_l[start:end]

                start = np.min(np.argwhere(mask_force_z_l_2 == True))
                end = np.max(np.argwhere(mask_force_z_l_2 == True))  # + swing_period_l

                force_x_l_2 = force_x[3][start:end]
                force_y_l_2 = force_y[3][start:end]
                force_z_l_2 = force_z[3][start:end]

                moment_x_l_2 = moment_x[3][start:end]
                moment_y_l_2 = -moment_y[3][start:end]
                moment_z_l_2 = -moment_z[3][start:end]

                # trajectory_x_l_2 = trajectory_x_r[start:end]
                # trajectory_y_l_2 = trajectory_y_l[start:end]
                trajectory_z_l_2 = trajectory_z_l[start:end]

                # ... Remove all zero GRF samples
                # if np.mean(force_z_2) <= FORCE_THRESHOLD or np.mean(force_z_3) <= FORCE_THRESHOLD \
                #     or np.mean(force_z_4) <= FORCE_THRESHOLD or np.mean(force_z_5) <= FORCE_THRESHOLD:
                #     n_bad_samples = n_bad_samples + 1
                #     continue

                # ... Resample
                force_x_r_1 = resample(force_x_r_1, N_SAMPLES, axis=0)
                force_x_l_1 = resample(force_x_l_1, N_SAMPLES, axis=0)
                force_x_r_2 = resample(force_x_r_2, N_SAMPLES, axis=0)
                force_x_l_2 = resample(force_x_l_2, N_SAMPLES, axis=0)

                force_y_r_1 = resample(force_y_r_1, N_SAMPLES, axis=0)
                force_y_l_1 = resample(force_y_l_1, N_SAMPLES, axis=0)
                force_y_r_2 = resample(force_y_r_2, N_SAMPLES, axis=0)
                force_y_l_2 = resample(force_y_l_2, N_SAMPLES, axis=0)

                force_z_r_1 = resample(force_z_r_1, N_SAMPLES, axis=0)
                force_z_l_1 = resample(force_z_l_1, N_SAMPLES, axis=0)
                force_z_r_2 = resample(force_z_r_2, N_SAMPLES, axis=0)
                force_z_l_2 = resample(force_z_l_2, N_SAMPLES, axis=0)

                moment_x_r_1 = resample(moment_x_r_1, N_SAMPLES, axis=0)
                moment_x_l_1 = resample(moment_x_l_1, N_SAMPLES, axis=0)
                moment_x_r_2 = resample(moment_x_r_2, N_SAMPLES, axis=0)
                moment_x_l_2 = resample(moment_x_l_2, N_SAMPLES, axis=0)

                moment_y_r_1 = resample(moment_y_r_1, N_SAMPLES, axis=0)
                moment_y_l_1 = resample(moment_y_l_1, N_SAMPLES, axis=0)
                moment_y_r_2 = resample(moment_y_r_2, N_SAMPLES, axis=0)
                moment_y_l_2 = resample(moment_y_l_2, N_SAMPLES, axis=0)

                moment_z_r_1 = resample(moment_z_r_1, N_SAMPLES, axis=0)
                moment_z_l_1 = resample(moment_z_l_1, N_SAMPLES, axis=0)
                moment_z_r_2 = resample(moment_z_r_2, N_SAMPLES, axis=0)
                moment_z_l_2 = resample(moment_z_l_2, N_SAMPLES, axis=0)

                # trajectory_x_r_1 = resample(trajectory_x_r_1, N_SAMPLES, axis=0)
                # trajectory_x_r_2 = resample(trajectory_x_r_2, N_SAMPLES, axis=0)
                # trajectory_x_l_1 = resample(trajectory_x_l_1, N_SAMPLES, axis=0)
                # trajectory_x_l_2 = resample(trajectory_x_l_2, N_SAMPLES, axis=0)

                # trajectory_y_r_1 = resample(trajectory_y_r_1, N_SAMPLES, axis=0)
                # trajectory_y_r_2 = resample(trajectory_y_r_2, N_SAMPLES, axis=0)
                # trajectory_y_l_1 = resample(trajectory_y_l_1, N_SAMPLES, axis=0)
                # trajectory_y_l_2 = resample(trajectory_y_l_2, N_SAMPLES, axis=0)

                trajectory_z_r_1 = resample(trajectory_z_r_1, N_SAMPLES, axis=0)
                trajectory_z_r_2 = resample(trajectory_z_r_2, N_SAMPLES, axis=0)
                trajectory_z_l_1 = resample(trajectory_z_l_1, N_SAMPLES, axis=0)
                trajectory_z_l_2 = resample(trajectory_z_l_2, N_SAMPLES, axis=0)

                # ... Standardize
                force_x_r_1 = standardize(force_x_r_1)
                force_x_l_1 = standardize(force_x_l_1)
                force_x_r_2 = standardize(force_x_r_2)
                force_x_l_2 = standardize(force_x_l_2)

                force_y_r_1 = standardize(force_y_r_1)
                force_y_l_1 = standardize(force_y_l_1)
                force_y_r_2 = standardize(force_y_r_2)
                force_y_l_2 = standardize(force_y_l_2)

                force_z_r_1 = standardize(force_z_r_1)
                force_z_l_1 = standardize(force_z_l_1)
                force_z_r_2 = standardize(force_z_r_2)
                force_z_l_2 = standardize(force_z_l_2)

                moment_x_r_1 = standardize(moment_x_r_1)
                moment_x_l_1 = standardize(moment_x_l_1)
                moment_x_r_2 = standardize(moment_x_r_2)
                moment_x_l_2 = standardize(moment_x_l_2)

                moment_y_r_1 = standardize(moment_y_r_1)
                moment_y_l_1 = standardize(moment_y_l_1)
                moment_y_r_2 = standardize(moment_y_r_2)
                moment_y_l_2 = standardize(moment_y_l_2)

                moment_z_r_1 = standardize(moment_z_r_1)
                moment_z_l_1 = standardize(moment_z_l_1)
                moment_z_r_2 = standardize(moment_z_r_2)
                moment_z_l_2 = standardize(moment_z_l_2)

                # trajectory_x_r_1 = standardize(trajectory_x_r_1)
                # trajectory_x_r_2 = standardize(trajectory_x_r_2)
                # trajectory_x_l_1 = standardize(trajectory_x_l_1)
                # trajectory_x_l_2 = standardize(trajectory_x_l_2)

                # trajectory_y_r_1 = standardize(trajectory_y_r_1)
                # trajectory_y_r_2 = standardize(trajectory_y_r_2)
                # trajectory_y_l_1 = standardize(trajectory_y_l_1)
                # trajectory_y_l_2 = standardize(trajectory_y_l_2)

                trajectory_z_r_1 = standardize(trajectory_z_r_1)
                trajectory_z_r_2 = standardize(trajectory_z_r_2)
                trajectory_z_l_1 = standardize(trajectory_z_l_1)
                trajectory_z_l_2 = standardize(trajectory_z_l_2)

                _features_l_1 = pd.DataFrame(
                    np.concatenate(
                        # [trajectory_x_l_1, trajectory_y_l_1, trajectory_z_l_1], axis=1
                        [trajectory_z_l_1], axis=1
                    ),
                    columns=[
                        # '0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x',
                        # '0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y',
                        # '0_z', '1_z', '2_z', '3_z', '4_z', '5_z', '6_z', '7_z', '8_z', '9_z', '10_z', '11_z'
                        # "fal_x",
                        # "tam_x",
                        # "fm5_x",
                        # "fm1_x",
                        # "fm2_x",
                        # "fal_y",
                        # "tam_y",
                        # "fm5_y",
                        # "fm1_y",
                        # "fm2_y",
                        "fal_z",
                        "tam_z",
                        "fm5_z",
                        "fm1_z",
                        "fm2_z",
                    ],
                )

                _features_l_2 = pd.DataFrame(
                    np.concatenate(
                        # [trajectory_x_l_2, trajectory_y_l_2, trajectory_z_l_2], axis=1
                        [trajectory_z_l_2], axis=1
                    ),
                    columns=[
                        # '0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x',
                        # '0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y',
                        # '0_z', '1_z', '2_z', '3_z', '4_z', '5_z', '6_z', '7_z', '8_z', '9_z', '10_z', '11_z'
                        # "fal_x",
                        # "tam_x",
                        # "fm5_x",
                        # "fm1_x",
                        # "fm2_x",
                        # "fal_y",
                        # "tam_y",
                        # "fm5_y",
                        # "fm1_y",
                        # "fm2_y",
                        "fal_z",
                        "tam_z",
                        "fm5_z",
                        "fm1_z",
                        "fm2_z",
                    ],
                )

                _features_r_1 = pd.DataFrame(
                    np.concatenate(
                        # [trajectory_x_r_1, trajectory_y_r_1, trajectory_z_r_1], axis=1
                        [trajectory_z_r_1], axis=1
                    ),
                    columns=[
                        # '0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x',
                        # '0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y',
                        # '0_z', '1_z', '2_z', '3_z', '4_z', '5_z', '6_z', '7_z', '8_z', '9_z', '10_z', '11_z'
                        # "fal_x",
                        # "tam_x",
                        # "fm5_x",
                        # "fm1_x",
                        # "fm2_x",
                        # "fal_y",
                        # "tam_y",
                        # "fm5_y",
                        # "fm1_y",
                        # "fm2_y",
                        "fal_z",
                        "tam_z",
                        "fm5_z",
                        "fm1_z",
                        "fm2_z",
                    ],
                )

                _features_r_2 = pd.DataFrame(
                    np.concatenate(
                        # [trajectory_x_r_2, trajectory_y_r_2, trajectory_z_r_2], axis=1
                        [trajectory_z_r_2], axis=1
                    ),
                    columns=[
                        # '0_x', '1_x', '2_x', '3_x', '4_x', '5_x', '6_x', '7_x', '8_x', '9_x', '10_x', '11_x',
                        # '0_y', '1_y', '2_y', '3_y', '4_y', '5_y', '6_y', '7_y', '8_y', '9_y', '10_y', '11_y',
                        # '0_z', '1_z', '2_z', '3_z', '4_z', '5_z', '6_z', '7_z', '8_z', '9_z', '10_z', '11_z'
                        # "fal_x",
                        # "tam_x",
                        # "fm5_x",
                        # "fm1_x",
                        # "fm2_x",
                        # "fal_y",
                        # "tam_y",
                        # "fm5_y",
                        # "fm1_y",
                        # "fm2_y",
                        "fal_z",
                        "tam_z",
                        "fm5_z",
                        "fm1_z",
                        "fm2_z",
                    ],
                )

                _features = pd.concat(
                    [_features_l_1, _features_l_2, _features_r_1, _features_r_2], axis=0
                )

                _features["subject"] = subject
                _features["velocity"] = velocity

                _target_r_1 = pd.DataFrame(
                    np.stack(
                        [
                            force_x_r_1,
                            force_y_r_1,
                            force_z_r_1,
                            moment_x_r_1,
                            moment_y_r_1,
                            moment_z_r_1,
                        ],
                        axis=1,
                    ),
                    columns=["fx", "fy", "fz", "mx", "my", "mz"],
                )

                _target_r_2 = pd.DataFrame(
                    np.stack(
                        [
                            force_x_r_2,
                            force_y_r_2,
                            force_z_r_2,
                            moment_x_r_2,
                            moment_y_r_2,
                            moment_z_r_2,
                        ],
                        axis=1,
                    ),
                    columns=["fx", "fy", "fz", "mx", "my", "mz"],
                )

                _target_l_1 = pd.DataFrame(
                    np.stack(
                        [
                            force_x_l_1,
                            force_y_l_1,
                            force_z_l_1,
                            moment_x_l_1,
                            moment_y_l_1,
                            moment_z_l_1,
                        ],
                        axis=1,
                    ),
                    columns=["fx", "fy", "fz", "mx", "my", "mz"],
                )

                _target_l_2 = pd.DataFrame(
                    np.stack(
                        [
                            force_x_l_2,
                            force_y_l_2,
                            force_z_l_2,
                            moment_x_l_2,
                            moment_y_l_2,
                            moment_z_l_2,
                        ],
                        axis=1,
                    ),
                    columns=["fx", "fy", "fz", "mx", "my", "mz"],
                )

                _target = pd.concat(
                    [_target_r_1, _target_r_2, _target_l_1, _target_l_2], axis=0
                )

                features = pd.concat([features, _features], axis=0, ignore_index=True)
                target = pd.concat([target, _target], axis=0, ignore_index=True)

                records.append(filename)

                n_samples = n_samples + 1

            except Exception as e:
                print(f"ERROR [{subject}/{velocity}/{filename}]: {e}")
                n_bad_samples = n_bad_samples + 1
                pass

            
        
    

print("Number of good samples: ", n_samples)
print("Number of bad samples: ", n_bad_samples)


  6%|▋         | 1/16 [01:03<15:52, 63.48s/it]

ERROR [Participant11/V1/T8.c3d]: zero-size array to reduction operation minimum which has no identity
ERROR [Participant11/V15/T6.c3d]: zero-size array to reduction operation minimum which has no identity


 31%|███▏      | 5/16 [05:20<11:26, 62.45s/it]

ERROR [Participant4/V1/T2.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T8.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T1.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T7.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T9.c3d]: zero-size array to reduction operation minimum which has no identity
ERROR [Participant4/V1/T4.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T5.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T3.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant4/V1/T6.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)


 38%|███▊      | 6/16 [06:13<09:55, 59.55s/it]

ERROR [Participant14/V3/T2.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T8.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T1.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T7.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T9.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T4.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T5.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T10.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T3.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V3/T6.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)
ERROR [Participant14/V1/T2.c3d]: zero-size array to reducti

 44%|████▍     | 7/16 [07:16<09:04, 60.55s/it]

ERROR [Participant14/V4/T6.c3d]: Shape of passed values is (1024, 4), indices imply (1024, 5)


100%|██████████| 16/16 [17:14<00:00, 64.64s/it]

Number of good samples:  980
Number of bad samples:  80


In [ ]:
features

In [ ]:
target

In [ ]:
x = features['fm1_z'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(x.shape[0]):
    plt.plot(x[i, :])

In [ ]:
y = target['fz'].to_numpy().reshape(-1, N_SAMPLES)
for i in range(y.shape[0]):
    plt.plot(y[i, :])

In [ ]:
path = '../../Dataset/Processed/data2_f15_t6_n1_all_vel.joblib'
data = {
    'X': features,
    'y': target
}
dump(data, path)

# Generating Train Test Sets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import load, dump
from sklearn.model_selection import train_test_split

N_SAMPLES = 1024
N_CHANNELS = 15

path = '../../Dataset/Processed/data2_f15_t6_n1_high_vel.joblib'
data = load(path)
features = data['X']
target = data['y']

# path1 = '../../Dataset/Processed/data2_f72_t3_n1_p1.joblib'
# path2 = '../../Dataset/Processed/data2_f72_t3_n1_p2.joblib'

# data1 = load(path1)
# data2 = load(path2)

# features1 = data1['X']
# features2 = data2['X']

# target1 = data1['y']
# target2 = data2['y']

# print(features1.shape)
# print(features2.shape)

# print(target1.shape)
# print(target2.shape)

In [ ]:
# x = features['11_z'].to_numpy().reshape(-1, N_SAMPLES)
# for i in range(1120, 1130):
#     plt.plot(x[i, :])

In [ ]:
# features = pd.concat([features1, features2])
# target = pd.concat([target1, target2])

# print(features.shape)
# print(target.shape)

In [ ]:
X = features.to_numpy().reshape(-1, N_SAMPLES, N_CHANNELS)
y = target.to_numpy().reshape(-1, N_SAMPLES, 6)

print(X.shape)
print(y.shape)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
path = '../../Dataset/Processed/data2_f15_t6_n1_high_vel_combined.joblib'
data = {
    'train_X': train_X,
    'test_X': test_X,
    'train_y': train_y,
    'test_y': test_y
}
dump(data, path)

<h1 align="center">Verification</h1>

In [ ]:
# path = '../../Dataset/Processed/data2_f15_t3_n1.joblib'
# data = load(path)

# features = data['X']
# target = data['y']

In [ ]:
# x = features['fal_x'].to_numpy().reshape(-1, N_SAMPLES)
# for i in range(x.shape[0]):
#     plt.plot(x[i, :])

In [ ]:
# y = target['fz'].to_numpy().reshape(-1, N_SAMPLES)
# for i in range(y.shape[0]):
#     plt.plot(y[i, :])

In [ ]:
# y = target['mz'].to_numpy().reshape(-1, N_SAMPLES)
# for i in range(y.shape[0]):
#     plt.plot(y[i, :])

In [ ]:
# features.shape